In [1]:
# %%
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import os, sys
import datetime
import math
import time
import numpy as np
import pandas as pd

In [13]:
options = webdriver.ChromeOptions() 
# options.add_argument('--headless')
options.add_experimental_option('excludeSwitches', ['disable-popup-blocking'])
options.add_argument('--window-size=1920x1080')
services = webdriver.chrome.service.Service() # CHROME_DRIVER_ADDRESS)

# %%
product_id = 40035209618
url = 'https://search.shopping.naver.com/catalog/' + str(product_id)
driver = webdriver.Chrome(service=services, options=options)
driver.get(url)

In [14]:
driver.find_elements(By.LINK_TEXT, '최신순')[0].click()
driver.find_elements(By.PARTIAL_LINK_TEXT, '쇼핑몰리뷰')[1].click()

In [15]:
reviews_list1 = []

while True:
    section_review = driver.find_element(By.ID, 'section_review')
    reviews_list = section_review.find_elements(By.XPATH, './ul/li')
    print(f"Num of found reviews: {len(reviews_list)}")

    for item0 in reviews_list:

        rating0 = item0.find_elements(By.XPATH, "div")[0].find_elements(By.XPATH, "span")[0].text[-1]
        vendor0 = item0.find_elements(By.XPATH, "div")[0].find_elements(By.XPATH, "span")[1].text
        reviewer0 = item0.find_elements(By.XPATH, "div")[0].find_elements(By.XPATH, "span")[2].text
        date0 = item0.find_elements(By.XPATH, "div")[0].find_elements(By.XPATH, "span")[3].text
        date1 = datetime.datetime.strptime(date0, "%y.%m.%d.")

        text0 = item0.find_elements(By.XPATH, "div")[1].find_element(By.XPATH, "div").find_element(By.XPATH, "em").text
        text1 = item0.find_elements(By.XPATH, "div")[1].find_element(By.XPATH, "div").find_element(By.XPATH, "p").text

        has_img = len(item0.find_elements(By.XPATH, "div")[1].find_elements(By.XPATH, "div")) - 1

        if has_img:
            images = item0.find_elements(By.XPATH, "div")[1].find_elements(By.XPATH, "div")[1].find_elements(By.XPATH, "span/img")
            images_url = [item.get_attribute("src").rstrip("?type=f287") for item in images]
        else:
            images_url = []

        reviews_list1.append([rating0, vendor0, reviewer0, date1, text0, text1, images_url])
    print(f"Current number of reviews: {len(reviews_list1)}")
    
    pagination_list = driver.find_elements(By.CLASS_NAME, "pagination_pagination__JW7zT")[0] # There are three. The last two are AiTEMS recommendation
    pages_list = pagination_list.find_elements(By.XPATH, "a")

    cur_page_idx = np.where(['now' in item.get_attribute('class') for item in pages_list])[0][0]
    cur_page_num = [item.get_attribute('data-nclick') for item in pages_list][cur_page_idx].split(':')[-1]
    print(f"Current page idx and num: {cur_page_idx}, {cur_page_num}")

    
    pages_list[cur_page_idx + 1].click() # When current page is 1, go to page 2
    time.sleep(5)

Num of found reviews: 20
Current number of reviews: 20
Current page idx and num: 0, 1
Num of found reviews: 20
Current number of reviews: 40
Current page idx and num: 1, 2
Num of found reviews: 20
Current number of reviews: 60
Current page idx and num: 2, 3
Num of found reviews: 20
Current number of reviews: 80
Current page idx and num: 3, 4
Num of found reviews: 20
Current number of reviews: 100
Current page idx and num: 4, 5
Num of found reviews: 20
Current number of reviews: 120
Current page idx and num: 5, 6
Num of found reviews: 20
Current number of reviews: 140
Current page idx and num: 6, 7
Num of found reviews: 20
Current number of reviews: 160
Current page idx and num: 7, 8
Num of found reviews: 20
Current number of reviews: 180
Current page idx and num: 8, 9
Num of found reviews: 20
Current number of reviews: 200
Current page idx and num: 9, 10
Num of found reviews: 20
Current number of reviews: 220
Current page idx and num: 1, 11
Num of found reviews: 20
Current number of re

IndexError: list index out of range

In [16]:
len(reviews_list1)

347

In [20]:
df_reviews0 = pd.DataFrame(reviews_list1)
df_reviews0.columns = ['평점', '판매자', '구매자', '날짜', '제목', '본문', '이미지URL']
df_reviews0

,평점,판매자,구매자,날짜,제목,본문,이미지URL
0,5,SK하이닉스공식대리점 도우,alvi****,2023-07-01,가볍고 빠르고 만족,가볍고 빠르고 만족,[]
1,5,SK하이닉스공식대리점 도우,puha****,2023-06-30,잇썹의 리뷰를 보고 구매 했는데 하이닉스 메모리 좋네요 ㅎㅎ 디자인도 독특하고 휴대...,잇썹의 리뷰를 보고 구매 했는데 하이닉스 메모리 좋네요 ㅎㅎ 디자인도 독특하고 휴대...,[https://phinf.pstatic.net/checkout/20230630_7...
2,5,SK하이닉스공식대리점 도우,leey****,2023-06-30,작고 속도도 빠르네요.잘 산듯 합니다.^^,작고 속도도 빠르네요.\n잘 산듯 합니다.^^,[]
3,5,SK하이닉스공식대리점 도우,gidr****,2023-06-30,작고 귀엽습니다다만 케이스는 뭔가 케이블이 안들어가는게 잘 안맞긴하네요,작고 귀엽습니다\n다만 케이스는 뭔가 케이블이 안들어가는게 잘 안맞긴하네요,[https://phinf.pstatic.net/checkout/20230630_4...
4,5,SK하이닉스공식대리점 도우,body****,2023-06-30,제품 아주 만족합니다.,제품 아주 만족합니다.,[https://phinf.pstatic.net/checkout/20230630_5...
...,...,...,...,...,...,...,...
342,5,SK하이닉스공식대리점 도우,whit****,2023-05-18,잇섭보고 삽니다.제품은 완전 만족함 완전 좋음 가격이 엄청 착해졌네요,잇섭보고 삽니다.\n제품은 완전 만족함 완전 좋음\n가격이 엄청 착해졌네요,[]
343,5,SK하이닉스공식대리점 도우,jaer****,2023-05-18,좋아요 작고 가벼워요,좋아요 작고 가벼워요,[]
344,5,SK하이닉스공식대리점 도우,ashl****,2023-05-18,"배송빠르고 좋습니다,","배송빠르고 좋습니다,",[]
345,5,SK하이닉스공식대리점 도우,upl9****,2023-05-18,대만족 합니다^^ :),대만족 합니다^^ :),[https://phinf.pstatic.net/checkout/20230518_2...


In [24]:
df_reviews0.to_excel('naver_x31_230702.xlsx', index = None)
df_reviews0.to_csv('naver_x31_230702.csv', encoding = 'utf-8-sig', index = None)
df_reviews0.to_pickle('naver_x31_230702.pickle')